<a href="https://colab.research.google.com/github/Avniiii2606/Weather-Prediction/blob/main/ImageRecougnition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pwd
!ls /content/drive/MyDrive/archive.zip

/content
/content/drive/MyDrive/archive.zip


In [ ]:
import zipfile
import os

zip_file_path='/content/drive/MyDrive/archive.zip'

# Define the directory to extract files to
train_images = '/content/extracted_files/'

# Create the directory if it doesn't exist
os.makedirs(train_images, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(train_images)

print(f'Files extracted to {train_images}')

Files extracted to /content/extracted_files/


In [ ]:
!unzip -q "/content/drive/MyDrive/path_to_your_zip_file.zip" -d "/content/extracted_files/"


unzip:  cannot find or open /content/drive/MyDrive/path_to_your_zip_file.zip, /content/drive/MyDrive/path_to_your_zip_file.zip.zip or /content/drive/MyDrive/path_to_your_zip_file.zip.ZIP.


In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to the dataset
dataset_path = '/content/extracted_files/dataset'

# Load image paths and labels
image_paths = []
labels = []

# Iterate through each weather condition folder
for label in os.listdir(dataset_path):
    label_path = os.path.join(dataset_path, label)
    if os.path.isdir(label_path):
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            # Check if the file is a valid image before appending (adjust extensions if needed)
            if image_name.lower().endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp')):
                image_paths.append(image_path)
                labels.append(label)
            else:
                print(f"Skipping non-image file: {image_path}")

# Check if any images were found
if not image_paths:
    print("Error: No image files found ")
else:
    # Convert labels to numerical format
    label_to_index = {label: index for index, label in enumerate(np.unique(labels))}
    labels = np.array([label_to_index[label] for label in labels])

    # Split data into training and test sets
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        image_paths, labels, test_size=0.2, random_state=42
    )

def load_and_preprocess_images(image_paths):
    images = []
    for image_path in image_paths:
        img = cv2.imread(image_path)
        if img is None:  # Check if image loaded successfully
            print(f"Error loading image: {image_path}")
            continue  # Skip to the next image
        img = cv2.resize(img, (128, 128))  # Resize to match your model input
        img = img / 255.0  # Normalize pixel values
        images.append(img)
    return np.array(images)

# Load training and test images
train_images = load_and_preprocess_images(train_paths)
test_images = load_and_preprocess_images(test_paths)

    # ... (Rest of your preprocessing code)

Skipping non-image file: /content/extracted_files/dataset/fogsmog/.ipynb_checkpoints
Skipping non-image file: /content/extracted_files/dataset/snow/.ipynb_checkpoints


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(label_to_index), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model with data augmentation
history = model.fit(
    datagen.flow(train_images, train_labels, batch_size=32),
    validation_data=(test_images, test_labels),
    epochs=30
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 11)                  │           1,419 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,306,059 (12.61 MB)

 Trainable params: 3,306,059 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 258s 1s/step - accuracy: 0.2979 - loss: 2.0483 - val_accuracy: 0.6086 - val_loss: 1.2051
Epoch 2/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.5232 - loss: 1.3807 - val_accuracy: 0.6385 - val_loss: 1.0690
Epoch 3/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 232s 1s/step - accuracy: 0.5607 - loss: 1.2704 - val_accuracy: 0.6501 - val_loss: 1.0418
Epoch 4/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 237s 1s/step - accuracy: 0.5813 - loss: 1.2474 - val_accuracy: 0.6195 - val_loss: 1.1219
Epoch 5/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 237s 1s/step - accuracy: 0.5892 - loss: 1.2083 - val_accuracy: 0.6603 - val_loss: 1.0023
Epoch 6/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 250s 1s/step - accuracy: 0.5978 - loss: 1.1825 - val_accuracy: 0.6676 - val_loss: 0.9995
Epoch 7/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 236s 1s/step - accuracy: 0.6174 - loss: 1.1407 - val_accuracy: 0.6727 - val_loss: 0.9564
Epoch 8/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - accuracy: 0.6238 - loss: 1.1187 - val_accu

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Test accuracy: {test_acc}')

43/43 ━━━━━━━━━━━━━━━━━━━━ 17s 375ms/step - accuracy: 0.7378 - loss: 0.7456
Test accuracy: 0.7463557124137878
